# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Вывод
Датасет состоит из 3 214 объектов и 5 признаков. Целевой признак для нашей задачи – is_ultra, т.е. модель, которую мы попытаемся построить, будет предсказывать значение 1 если клиенту нужно предложить тариф "Ультра" или 0 если тариф "Смарт". В этом и заключается задача бинарной классификации, т.к. наш целевой признак является категориальным.

## 2. Разбейте данные на выборки

In [8]:
# 60% отводим под обучающую выборку
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
# половину из оставшихся 40% отдаем на валидационную, другую половину – на тестовую
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

### Вывод
В итоге получили следующее соотношение: 60% | 20% | 20%

Чуть позднее, после проверки модели на валидационной выборке выполним оценку ещё и на тестовом наборе. Это позволит правильно оценить готовую модель.

## 3. Исследуйте модели

Для решения задачи классификации рассмотрим следующие изученные модели:

- дерево решений / decision tree
- случайный лес / random forest
- логистическую регрессию / logistic regression

In [9]:
#Подготовим фичи и целевые признаки обучающей и валидационных выборок
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [10]:
# Функция, рассчитывающая оценку правильности на валидационной выборке для обученной модели
def accuracy_score_valid(model):
    predictions_valid = model.predict(features_valid)
    return accuracy_score(target_valid, predictions_valid)

## Дерево решений / Decision Tree

In [12]:
# Посмотрим как глубина решающего дерева влияет на оценку правильности
for max_depth in range(1, 21, 2):
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=12345)
    model.fit(features_train, target_train)
    print(f"max_depth = {max_depth}:\t{accuracy_score_valid(model)}")

max_depth = 1:	0.7542768273716952
max_depth = 3:	0.7853810264385692
max_depth = 5:	0.7791601866251944
max_depth = 7:	0.7822706065318819
max_depth = 9:	0.7822706065318819
max_depth = 11:	0.7620528771384136
max_depth = 13:	0.7558320373250389
max_depth = 15:	0.7465007776049767
max_depth = 17:	0.7356143079315708
max_depth = 19:	0.7278382581648523


### Выводы
Судя по валидационной выборке, дерево решений имеет самую высокую оценку правильности 0.7869362363919129, когда задан гиперпараметр глубины равный 7.

## Случайный лес / Random Forest

In [13]:
# В качестве гиперпараметра глубины дерева возьмем значение, найденное для предыдущей модели.
# А количество деревьев для нашего случайного леса будет искать в диапазоне от 10 до 100 с шагом 10.
for estim in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=estim, max_depth=7, random_state=12345)
    model.fit(features_train, target_train)
    print(f"n_estimators = {estim}:\t{accuracy_score_valid(model)}")

n_estimators = 10:	0.7947122861586314
n_estimators = 20:	0.8009331259720062
n_estimators = 30:	0.80248833592535
n_estimators = 40:	0.80248833592535
n_estimators = 50:	0.80248833592535
n_estimators = 60:	0.7993779160186625
n_estimators = 70:	0.8009331259720062
n_estimators = 80:	0.7993779160186625
n_estimators = 90:	0.7993779160186625
n_estimators = 100:	0.8009331259720062


### Выводы
Модель случайного леса предсказывает тариф точнее, но, как мы видим, не на много – 0.8102643856920684, даже при количестве деревьев леса равным 30

## Логистическая регрессия / Logistic Regression

In [17]:
def logistiRegres(solver):  
    model = LogisticRegression(solver = solver, random_state=12345)
    model.fit(features_train, target_train)
    return accuracy_score_valid(model)

In [18]:
logistiRegres('liblinear')

0.7589424572317263

Логистическая регрессия на валидационной выборке дала самые худшие результаты. Попробуем их улучшить изменив гиперпараметры, например, solver – алгоритм задачи оптимизации.

In [20]:
solvers = ['newton-cg', 'lbfgs']
for solver in solvers:
    print(f'{solver}: {logistiRegres(solver=solver)}')

newton-cg: 0.7558320373250389
lbfgs: 0.7107309486780715


/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


### Вывод
Даже используя другой алгоритм для решения задачи оптимизации модель логистической регрессии менее точна, чем дерево решений и случайный лес.

## 4. Проверьте модель на тестовой выборке

В результате наших экспериментов на валидационной выборке мы выяснили, что самую высокую оценку правильности дает лес решений с глубиной равной 7 и количеством деревьев равным 30.

Проверим как поведет себя модель на тестовой выборке.

In [21]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

model = RandomForestClassifier(n_estimators=30, max_depth=7, random_state=12345)
model.fit(features_train, target_train)

predictions_test = model.predict(features_test)
accuracy_score(target_test, predictions_test)

0.8040435458786936

### Выводы
На тестовой выборке наша модель случайного леса показывает приближенное к тренировочной выборке значение accuracy. Таким образом, мы не видим здесь явной проблемы с переобучением, и можем считать, что нам удалось построить модель для задачи бинарной классификации, которая выберет подходящий тариф.

## 5. (бонус) Проверьте модели на адекватность

Мы хотим, чтобы выпущенная в продакшен модель не была подвержена переобучению, т.е. работала хорошо на данных, которые она не видела в процессе обучения.

Проверка модели на адекватность sanity check – нетривиальная задача. Для разных моделей машинного обучения существуют разные подходы. Выбор же ошибочной метрики качества может ввести в заблуждение. Например, мы не можем полагаться только на accuracy для задач классификации, особенно в ситуации с дисбалансом классов: когда одна метка встречается чаще чем другая; или же когда в обучающей и валидационных выборках оказываются дубликаты.

In [22]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Для задач бинарной классификации могут быть использованы следующие методы:

- Матрица ошибок / confusion matrix
- Тесты бинарной классификации / binary classification tests
- Коэффициент конверсии / conversion rates
- ROC-кривая / ROC curve
- Совокупный доход / cumulative gain
- Lift-кривая / lift chart

Работает и сравнение модели со случайной.

In [23]:
df_test['is_ultra'].value_counts()

0    440
1    203
Name: is_ultra, dtype: int64

Доля большего класса тестовой выборки равна 

In [25]:
temp = df_test['is_ultra'].value_counts()
temp[0]/(temp.sum()/100)

68.42923794712287

полученная нами модель имеет accuracy ~0.8. Таким образом, мы можем считать модель адекватной для использования в продакшене.

Обученная модель должна быть лучше предположения что дисбаланс классов в обучающей выборке будет таким же и на тестовой выборке.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 4: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
